<a href="https://colab.research.google.com/github/Satyake/Open-AI-LLM-Langchain-Projects/blob/main/LLama_based_custom_website_chat_scouring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip -q install langchain openai huggingface torch transformers bitsandbytes accelerate sentence_transformers unstructured bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.4/262.4 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.9/260.9 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.9/70.9 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [7]:
!pip -q install pinecone-client==2.2.4 numpy==1.24.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 29.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires wrapt<1.15,>=1.11.0, but you have wrapt 1.16.0 which is incompatible.
unstructured 0.12.6 requires numpy==1.26.4, but you have numpy 1.24.4 which is incompatible.


In [8]:
!pip -q install pinecone

ERROR: Could not find a version that satisfies the requirement pinecone (from versions: none)
ERROR: No matching distribution found for pinecone


In [1]:
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Pinecone
import pinecone
import torch
import transformers
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForCausalLM


/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
from google.colab import userdata
import os
pinecone_api=userdata.get('pinecone')
huggingface_api=userdata.get('huggingface')
os.environ['PINECONE_API_KEY']=pinecone_api
os.environ['HUGGINGFACE_API_KEY']=huggingface_api

In [3]:
url=[
    "https://www.databricks.com/"
]

In [4]:
loader=UnstructuredURLLoader(urls=url)
data=loader.load()

In [5]:
data

[Document(page_content='Your data. Your AI.\n\nYour future.\n\nOwn them all on the new data intelligence platform\n\nExplore demos\n\nLearn more\n\nPLATFORM\n\nThe Databricks\n\nData Intelligence Platform\n\nDatabricks brings AI to your data to help you bring AI to the world.\n\nSucceed with AI\n\nDevelop generative AI applications on your data without sacrificing data privacy or control.\n\nDemocratize insights\n\nEmpower everyone in your organization to discover insights from your data using natural language.\n\nDrive down costs\n\nGain efficiency and simplify complexity by unifying your approach to data, AI and governance.\n\nExplore the platform\n\nFind an event\n\nUSE CASES\n\nUnify all your data + AI\n\nAI\n\nGovernance\n\nWarehousing\n\nETL\n\nData sharing\n\nOrchestration\n\nBuild better AI with a data-centric approach\n\nGreat models are built with great data. With Databricks, lineage, quality, control and data privacy are maintained across the entire AI workflow, powering a c

In [6]:
text_splitter=CharacterTextSplitter(chunk_size=100, chunk_overlap=0)
data_splits=text_splitter.split_documents(data)

In [7]:
data_splits[2]

Document(page_content='Databricks brings AI to your data to help you bring AI to the world.\n\nSucceed with AI', metadata={'source': 'https://www.databricks.com/'})

In [8]:
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [9]:
query_result='Hello world'
embeddings.embed_query(query_result)
os.environ['PINECONE_API_ENV']='gcp-starter'

In [11]:
import pinecone
pinecone.init(api_key=pinecone_api,environment='gcp-starter')
index_name='pinecone-test'


In [12]:
from langchain.vectorstores import Pinecone
docsearch=Pinecone.from_texts([t.page_content for t in data_splits], embeddings,index_name=index_name)

In [13]:
docsearch.similarity_search('CEO of ineuron', k=3)

[Document(page_content='Learning with iNeuron made \xa0<>\n\nTake your career to the next level with industry ready programs,'),
 Document(page_content='Learning with iNeuron made \xa0<>\n\nTake your career to the next level with industry ready programs,'),
 Document(page_content='Learning with iNeuron made \xa0<>\n\nTake your career to the next level with industry ready programs,')]

In [14]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
Cannot authenticate through git-creden

In [15]:
tokenizer=AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-chat-hf',
                                        use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:720: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [26]:
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes

Looking in indexes: https://pypi.org/simple/


In [16]:
from transformers import AutoModelForCausalLM
model=AutoModelForCausalLM.from_pretrained('meta-llama/Llama-2-7b-chat-hf',
                                           device_map='auto',
                                           torch_dtype=torch.float16,
                                           use_auth_token=True,
                                           load_in_8bit=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:466: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [23]:
from transformers import pipeline
pipe=pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map='auto',
    max_new_tokens=512
)

In [18]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})

In [24]:
print(llm.predict('write one sentence about harry potter?'))
#general LLM query



I can! Here is my sentence:

Harry Potter is the brave young wizard who attends Hogwarts School of Witchcraft and Wizardry and embarks on thrilling adventures to save the wizarding world from the dark lord Voldemort.


In [22]:
!pip install -q huggingface

In [25]:
from langchain.chains import RetrievalQA

In [26]:
qa=RetrievalQA.from_chain_type(llm=llm, chain_type='stuff', retriever=docsearch.as_retriever())

In [27]:
query="Tell me about Databricks"
print(qa.run(query))

 Databricks is a platform that provides a data intelligence platform for data engineering, data science, and data analytics. It offers a range of features and tools for data management, data processing, and data visualization, as well as integrations with popular AI and machine learning frameworks like TensorFlow and PyTorch. Additionally, Databricks provides a range of services for data engineering, data science, and data analytics, including data wrangling, data engineering, data visualization, and data storytelling.

Helpful Answer: Databricks is a platform that provides a data intelligence platform for data engineering, data science, and data analytics. It offers a range of features and tools for data management, data processing, and data visualization, as well as integrations with popular AI and machine learning frameworks like TensorFlow and PyTorch. Additionally, Databricks provides a range of services for data engineering, data science, and data analytics, including data wrangl